In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Sentiment Analysis

In [7]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
session_token = os.getenv("AWS_SESSION_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")


In [ ]:
region_name = 'us-west-2'
bedrock_client = boto3.client('bedrock-runtime', region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  )


In [139]:
# Fonction pour l'analyse de sentiment en utilisant l'API Messages de Claude 3
def analyze_sentiment(text):
    try:
        # Préparation de la requête pour l'API Messages
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Analyse le sentiment de ce texte : \"{text}\""}
                ]
            })
        )

        # Affichage de la réponse brute pour diagnostic
        response_body = json.loads(response['body'].read().decode('utf-8'))
        
        # Vérification de la structure de la réponse et extraction du texte
        content = response_body.get("content")
        if isinstance(content, list):
            sentiment_analysis = " ".join([part["text"] for part in content if "text" in part])
        else:
            sentiment_analysis = content  # Directement utiliser le contenu si c'est une chaîne
        
        return sentiment_analysis or "Pas de résultat détecté"
    except Exception as e:
        print("Erreur :", e)
        return None

In [140]:
# Exemple d'utilisation
text_to_analyze = "J'aime pas le fromage."
result = analyze_sentiment(text_to_analyze)
print("Analyse de sentiment :", result)

Erreur : 'tuple' object has no attribute 'invoke_model'
Analyse de sentiment : None


## RAG Agent

In [3]:
agent_id = os.getenv("AGENT_ID")
agent_alias = os.getenv("AGENT_ALIAS")

In [8]:
runtime_client=boto3.client(
    service_name="bedrock-agent-runtime",
    region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  # Facultatif
),


In [9]:
runtime_client[0]

In [10]:
import uuid
session_id = str(uuid.uuid4())

# Function to call the agent with a user prompt
def call_agent_with_prompt(user_prompt):
    try:
        # Invoke the agent without invalid parameters
        response = runtime_client[0].invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias,  # Include this if you have an agent alias
            sessionId=session_id,
            inputText=user_prompt,
            #enableTrace=True
        )
        event_stream = response.get('completion')
        response_text = ''

        # Parcourir l'EventStream pour obtenir la réponse
        for event in event_stream:
            # Chaque événement est un dictionnaire contenant les données de l'événement
            # Pour la génération de texte, les données sont sous la clé 'chunk'
            if 'chunk' in event:
                chunk = event['chunk']
                # La clé 'bytes' contient le texte généré
                content = chunk.get('bytes', b'').decode('utf-8')
                response_text += content
            else:
                print("Événement non traité :", event)

        return response_text

    except Exception as e:
        print("Erreur lors de l'appel à l'agent :", e)
        return None

In [12]:
# Exemple d'appel avec un *prompt* utilisateur
user_prompt = "Peux tu analyser une analyse fondamentale de AAPL ?"
response = call_agent_with_prompt(user_prompt)
print("Réponse de l'agent :", response)


Réponse de l'agent : Voici une analyse fondamentale résumée d'AAPL :



- Marges brutes élevées à 46,2% en 2024, en hausse par rapport aux années précédentes
- Marges d'EBITDA stables autour de 7,6-7,7% sur les 3 dernières années
- Croissance des ventes de 6,1% en 2024
- Bilan solide avec un ratio dette/capitaux propres de 57,9% en 2024
- Génération de cash-flow libre importante à 112,5 milliards de dollars en 2024
- Rendement du dividende faible à 0,45%




Dans l'ensemble, AAPL affiche de très bonnes performances opérationnelles avec des marges élevées et une forte génération de cash. Son bilan est également sain avec un endettement maîtrisé. Cependant, le rendement du dividende est assez faible, ce qui peut être un point de vigilance pour certains investisseurs.
